<a href="https://colab.research.google.com/github/keerthucit/EIP2Assignment/blob/master/Copy_of_4a_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# https://keras.io/
!pip install -q keras
import keras


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import sgd
from keras import metrics
#images = np.random.randint(0, 50, (4, 128, 128, 3), dtype=np.uint8)
from keras_preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
#from django_pg import models
from keras import backend as K
import tensorflow as tf
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import SGD
import numpy as np
import sklearn.metrics as metrics
import h5py


In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
import tensorflow as tf
from keras import backend as k
import os
# Hyperparameters
batch_size = 64
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'
#batch_size = 128
num_classes = 10
epochs = 30
l = 100
#num_filter = 12
num_filter = 16
#num_filter=64
#compression = 0.5
compression=1.0
dropout_rate = 0.0
random_rotation = image.random_rotation
random_shift = image.random_shift
random_shear = image.random_shear
random_zoom = image.random_zoom
apply_channel_shift = image.apply_channel_shift
random_channel_shift = image.random_channel_shift
apply_brightness_shift = image.apply_brightness_shift
random_brightness = image.random_brightness
apply_affine_transform = image.apply_affine_transform
load_img = image.load_img


In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)




In [0]:
# Dense Block0:
def add_denseblock(input, nb_dense_block=4, growth_rate=32, num_filter=16, reduction=0.0, dropout_rate=0.0, weight_decay=1e-4, classes=1000, weights_path=None):
#def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 16, dropout_rate = 0.0):
#def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    #avg = MaxPooling2D(pool_size=2, strides=2)(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
   #AvgPooling = MaxPooling2D(pool_size=2, strides=2)(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 16
#num_filter = 64
dropout_rate = 0.0
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [0]:
def update_model(x_train, y_train,batch_size=64,epochs=30,verbose=1,
          validation_data=(x_test, y_test),validation_split=0.2,
          ):
  
       #X, y = train[:, :-n_seq], train[:, -n_seq:]
        #X = X.reshape(X.shape[0], n_lag, n_features)           
        for i in range(updates):              
            #model.fit(X, y,epoch=1,b_size=1,verbose=0, shuffle=False)
            model.fit(x_train, y_train,batch_size=64,epochs=30,verbose=1,
          validation_data=(x_test, y_test),validation_split=0.2,
         )
            model.reset_states()
        return model
 

In [0]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_53 (Conv2D)              (None, 32, 32, 16)   432         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_53 (BatchNo (None, 32, 32, 16)   64          conv2d_53[0][0]                  
__________________________________________________________________________________________________
activation_53 (Activation)      (None, 32, 32, 16)   0           batch_normalization_53[0][0]     
__________________________________________________________________________________________________
conv2d_54 

In [0]:
# determine Loss function and Optimizer
#model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
sgd=SGD(lr=1e-1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

#model.compile(loss='mean_squared_error',optimizer='sgd',metrics=['accuracy'])
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


In [0]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        #featurewise_center=False,  # set input mean to 0 over the dataset
        #samplewise_center=False,  # set each sample mean to 0
        #featurewise_std_normalization=False,  # divide inputs by std of the dataset
        #samplewise_std_normalization=False,  # divide each input by its std
        #zca_whitening=False,  # apply ZCA whitening
        #zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        #shear_range=0.,  # set range for random shear
        #zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.1,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        #fill_mode='nearest',
        #cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True)  # randomly flip images
        #vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        #rescale=None,
        # set function that will be applied on each input
        #preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        #data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        #validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    


Using real-time data augmentation.


In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

SyntaxError: ignored

In [0]:
#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)--leave this

# Fit the model on the batches generated by datagen.flow().--leave this
#model.fit(x_train, y_train,
                   #batch_size=batch_size,
                   #epochs=epochs,
                   #verbose=1,
                   #validation_data=(x_test, y_test),
                   #validation_split=0.2,
                   #callbacks=[reduce_lr])

#filepath="/tmp/4a_weights.hdf5"
#filepath="/content/gdrive/My Drive/2times:ep:dp:{epoch:03d}-{val_acc:.3f}.hdf5"
#model.fit_generator(datagen.flow(x_train, y_train,batch_size=batch_size),steps_per_epoch=len(x_train) // batch_size, epochs=epochs,
#                   callbacks=callbacks,validation_data=(x_test, y_test))

#from keras.callbacks import EarlyStopping, ModelCheckpoint
#from keras.models import load_model


#earlyStopping=EarlyStopping(monitor='val_loss', patience=8, verbose=1, mode='max', min_delta=0)
#checkp=ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)

#model.fit(X_train, Y_train, batch_size=512, nb_epoch=100, verbose=1, 
        # callbacks=[earlyStopping, checkp], 

#model = load_model(filepath='best_model.h5')
#--------------   

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=1e-5)
# checkpoint
filepath="/tmp/PreTrain_weights-improvement-{epoch:03d}-{val_acc:.3f}.hdf5"
#filepath="/tmp/4a_weights.hdf5"
model_checkpoint =keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss',save_weights_only=True,save_best_only=True, verbose=1, mode='max')
callbacks_list = [reduce_lr,model_checkpoint]
model.fit(x_train, y_train,batch_size=64, initial_epoch=0, epochs=30, verbose=1, validation_data=(x_test, y_test), validation_split=0.2,callbacks=callbacks_list)
#checkpointer = keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
#model.fit(x_train, y_train, batch_size=64,initial_epoch=0,epochs=20, verbose=0, validation_data=(x_test, y_test), callbacks=[checkpointer])



RuntimeError: ignored

In [0]:
!fusermount -u drive
!google-drive-ocamlfuse drive


In [0]:
!pwd

/content


In [0]:
ls 

4a_model.h5  adc.json  chkpoint/  drive/  My_CheckPoints/  sample_data/


In [0]:
!ls chkpoint

Mychkpoint.hd5


In [0]:
from keras.models import model_from_yaml

yaml_string = model.to_yaml()
model = model_from_yaml(yaml_string)

In [0]:
from keras.models import Sequential
# Save the weights
model.save_weights('./chkpoint/Mychkpoint.hd5')
# Restore the weights
model = create_model()
model.load_weights('./chkpoint/Mychkpoint.hd5')

NameError: ignored

In [0]:

loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

NameError: ignored

In [0]:

#from keras.models import load_model

#model.load_weights(filepath, by_name=False)
model.save('my_model.h5')
new_model = keras.models.load_model('my_model.h5')

/usr/local/lib/python3.6/dist-packages/keras/models.py:282: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [0]:
#To Save the model
#model.save('4a_model.h5')
#model_file=drive.createFile({'title' : '4a_Model.h5'})
#model_file.SetContentFile('4a_Model.h5')
#model_file.Upload()
#download to GDrive
#drive.CreateFile('4a_Model.h5')
# Save the trained weights in to .h5 format
#model.save_weights("DNST_model.h5")
#print("Saved model to disk")

NameError: ignored

In [0]:

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])
 

RuntimeError: ignored

In [0]:
#Next30 epochs
model = Model(inputs=[output], outputs=[outputnext1])
model.summary()

NameError: ignored

In [0]:
# Next 30 Epochs SGD 

sgd=SGD(lr=1e-2, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

#model.compile(loss='mean_squared_error',optimizer='sgd',metrics=['accuracy'])
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [0]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(        
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,       
        height_shift_range=0.1,        
        channel_shift_range=0.1,  # set range for random channel shifts
        horizontal_flip=True
    rotation_range=360)      
#width_shift_range=0.05,height_shift_range=0.05,shear_range=0.10,
#zoom_range=0.3,rescale=1./255,horizontal_flip=True,vertical_flip=True)
    # randomly flip images  
        datagen.fit(x_train)


In [0]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=np.sqrt(0.1), cooldown=0, 
                              patience=5, min_lr=1e-5)
filepath="/content/gdrive/My Drive/2times:ep:dp:{epoch:03d-{val_acc:.3f}hdf5"
#filepath="/tmp/4a_weights.hdf5"
model_checkpoint = ModelCheckpoint(filepath, monitor='val_acc', save_weights_only=True,
                                   save_best_only=True, verbose=1)

callbacks = [reduce_lr ,model_checkpoint]

model.fit_generator(datagen.flow(x_train, y_train,batch_size=batch_size),                    steps_per_epoch=len(x_train) // batch_size, epochs=epochs,
                    callbacks=callbacks,validation_data=(x_test, y_test))


In [0]:
plot_model_history(model_info)
print("Model takes %0.2f seconds to train" %(end - start))
#Test Accuracy
print("Accuracy on Test Data %0.2f" %accuracy(x_test, y_test, model))

NameError: ignored

In [0]:

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

RuntimeError: ignored

In [0]:
#from google.colab import files

#files.download('DNST_model.h5')

In [0]:
print ((model.get_weights()))
